In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
from IPython.display import clear_output
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from datasets import Dataset, load_dataset, load_from_disk
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from evaluate import load
import datetime
import os
# from sklearn.model_selection import train_test_split


2025-01-25 14:24:16.396274: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-25 14:24:18.774474: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-25 14:24:19.496319: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-25 14:24:23.579000: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-25 14:24:50.601976: W tensorflow/compiler/tf2

In [2]:
#TODO: change to work with classes?
NUCLEPTIDE_TRANSFORMER_500M_1000G_NAME = "InstaDeepAI/nucleotide-transformer-500m-1000g"
INSTADEEP_BASIC_RUN_MODELS = [NUCLEPTIDE_TRANSFORMER_500M_1000G_NAME]
INSTADEEP_KMER_SIZE = 6
INSTADEEP_BASIC_MAX_TOKEN_LENGTH = 1000

#TODO: Change so that these values are taken from JSON
TRAIN_PATH = "table_browser_hg38_unmasked_CpG_train.csv"
TEST_PATH = "table_browser_hg38_unmasked_CpG_test.csv"
MODEL_NAME = NUCLEPTIDE_TRANSFORMER_500M_1000G_NAME
DO_DOWNSAMPLING = False
LABLES_USED = [0,1]
TRAIN_SIZE = 1
BATCH_SIZE = 1
SUFFIX = "unmasked_cpg"
EPOCH_NUM = 10
METRIC = "None"
CONTINUE_FROM = ""
DATASET_TRAIN_SAVE_PATH = "unmasked_cpg_train"
DATASET_TEST_SAVE_PATH = "unmasked_cpg_test"

In [3]:
# train_df

In [3]:
train_df = pd.read_csv(TRAIN_PATH)
train_df = train_df.head(int(len(train_df) *TRAIN_SIZE))
test_df = pd.read_csv(TEST_PATH)

In [4]:
def create_INSTADEEP_basic_label_seq_from_df(df):
    sequences = []
    labels = []
    for row_idx, row in df.iterrows():
        if row_idx % 100 == 0:
            clear_output(wait=True)
            print(str(int(100*(row_idx / len(df)))) + "%")

        curr_label = row["labels"]
        # for i, seq in enumerate(row["result"]):
        #     curr_label.extend([row["stags"][i]]*len(seq))
        
        label = np.zeros(len(curr_label)//INSTADEEP_KMER_SIZE,dtype=np.int64)
        for i in range(0,len(curr_label),INSTADEEP_KMER_SIZE):
            
            for j in range(1,len(LABLES_USED)):
                # TODO: consinder changing str(j) to something else
                if i+INSTADEEP_KMER_SIZE <len(curr_label) and str(j) in curr_label[i:i+INSTADEEP_KMER_SIZE]:
                    label[i//INSTADEEP_KMER_SIZE:i//INSTADEEP_KMER_SIZE+1] = j
                    break
            else: # unneeded
                label[i//INSTADEEP_KMER_SIZE:i//INSTADEEP_KMER_SIZE+1] = 0
        # seq_str = "".join(row["result"])[:len(curr_label)]
        seq_str = row["seq"]

        # TODO: check why sometimes seq length is not 1000
        # TODO: check why sometimes seq is type float
        if (len(str(seq_str))) != 6000 or (len(label)) != 1000: 
            continue
        sequences.append(seq_str)
        labels.append(label)
    return sequences, labels



In [5]:
train_sequences,train_labels = create_INSTADEEP_basic_label_seq_from_df(train_df)
test_sequences, test_labels = create_INSTADEEP_basic_label_seq_from_df(test_df)

99%


In [6]:
train_df.head(2)["seq"]

0    AGGTAGCTCTACTGCCTCCTCTTAAAACCAACAAAGGAAAGAGAGA...
1    CAGCCTGGGCGACAGAGTGAGTCTAAAAAAAATAAAAAAGGAATTC...
Name: seq, dtype: object

In [7]:
def create_datasets_instadeep_basic(seq,labels,tokenizer):
    tokenized_dataset = tokenizer(
        seq,
        max_length=INSTADEEP_BASIC_MAX_TOKEN_LENGTH,
        padding='max_length',
        truncation=True
    )
    dataset = Dataset.from_dict(tokenized_dataset)
    dataset = dataset.add_column("labels", labels)
    return dataset
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
train_dataset = create_datasets_instadeep_basic(train_sequences,train_labels,tokenizer)
test_dataset = create_datasets_instadeep_basic(test_sequences, test_labels,tokenizer)
#TODO: change so if already saved to disk, load existing


In [8]:
TRAIN_PATH

'table_browser_hg38_unmasked_CpG_train.csv'

In [9]:
# TODO: check if resaving changes results
train_dataset.save_to_disk(DATASET_TRAIN_SAVE_PATH)
test_dataset.save_to_disk(DATASET_TEST_SAVE_PATH)

Saving the dataset (0/1 shards):   0%|          | 0/33423 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/11138 [00:00<?, ? examples/s]

In [11]:
dic = {}
for i in range(len(train_dataset)):
    if i % 100 == 0:
            clear_output(wait=True)
            print(str(int(100*(i / len(train_dataset)))) + "%")
    for val in train_dataset[i]["labels"]:
        if val not in dic:
            dic[val] = 0
        dic[val]+= 1
for label in dic:
    print(f"{label} label:",dic[label])

99%
0 label: 30231385
1 label: 3229615


In [12]:
print("a")

a


In [13]:
model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME,num_labels=len(LABLES_USED), device_map="auto")
data_collator = DataCollatorForTokenClassification(tokenizer)

Some weights of EsmForTokenClassification were not initialized from the model checkpoint at InstaDeepAI/nucleotide-transformer-500m-1000g and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
batch_size = BATCH_SIZE
# TODO: USE their parameters and LORA. copy from them
args = TrainingArguments(
    f"{MODEL_NAME}-{SUFFIX}",
    evaluation_strategy = "no",
    do_eval=False,
    # evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=3*batch_size,
    #per_device_eval_batch_size=batch_size,
    num_train_epochs=EPOCH_NUM,
    weight_decay=0.001,
    # load_best_model_at_end=True,
    metric_for_best_model=METRIC,
    push_to_hub=False,
)

/sci/nosnap/michall/roeizucker/new_python_env/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


/tmp/ipykernel_212993/2344023472.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
if CONTINUE_FROM == "":
    trainer.train()
else:
    trainer.train(resume_from_checkpoint=CONTINUE_FROM)

Step,Training Loss
500,0.179800
1000,0.165600
1500,0.147700
2000,0.132800
2500,0.133600
3000,0.123600
3500,0.122000
4000,0.124500
4500,0.107600
5000,0.107300
